# Get the content from LOTR-Wiki

Let's use the site map instead of crawling classically :)

In [ ]:
import requests
import xml.etree.ElementTree as ET

sitemaps = """https://lotr.fandom.com/sitemap-newsitemapxml-NS_0-id-2-43291.xml
https://lotr.fandom.com/sitemap-newsitemapxml-NS_0-id-43291-55198.xml
https://lotr.fandom.com/sitemap-newsitemapxml-NS_14-id-21-55165.xml""".split("\n")

r = requests.get(sitemaps[0])
root = ET.fromstring(r.text)
print(root.tag, root.tail, root.attrib, f"Has {len([0 for child in root])} children" )


In [ ]:
def pre_order(root:ET.Element, indents=0):
    print("\t"*indents, root.text)
    for child in root:
        pre_order(child, indents+1)

pre_order(root)


Alright, so a bunch of stuff but site links are separated from everything else by newlines. let's use that fact to get a list of all sites and count how many there are.

In [ ]:
sites = []
def fill_sites(root:ET.Element):
    sites.extend(list(filter(lambda x : "https" in x, root.text.split("\n"))))
    for child in root:
        fill_sites(child)

for sitemap in sitemaps:
    r = requests.get(sitemap)
    root = ET.fromstring(r.text)
    fill_sites(root)
len(sites), sites


In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.content, "html.parser")
links = soup.find_all("a")
links

In [ ]:
for link in links:
    print(link['href'])

# Set up and fill the RDB

In [ ]:
import sqlite3 as sql

conn = sql.connect('LOTR.db')